<a href="https://colab.research.google.com/github/hyndhavi11/OIBSIP/blob/main/chatgptreview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reviews are actually one of the best data-driven strategies through which you can improve your product or services. It helps you to understand what people think about your product, what they dislike about it, and what more they expect from it. So, if you want to know how to analyze reviews to understand what people think about your product, this article is for you. In this article, I'll take you through the task of analyzing reviews for ChatGPT using Python; it will also be able to help us understand how we can analyze reviews to understand what people think about ChatGPT.

#**ChatGpt Reviews Analysis with Python**

I have a dataset of end-users' reviews of chatbots with textual feedback, ratings, and dates of review. Download this dataset from here /content/chatgpt_reviews.csv.

**Import Neccessary Libraries:**

In [16]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"


In [17]:
#load the dataset
data = pd.read_csv("/content/chatgpt_reviews.csv")
data.head()

,Review Id,Review,Ratings,Review Date
0,6fb93778-651a-4ad1-b5ed-67dd0bd35aac,good,5,2024-08-23 19:30:05
1,81caeefd-3a28-4601-a898-72897ac906f5,good,5,2024-08-23 19:28:18
2,452af49e-1d8b-4b68-b1ac-a94c64cb1dd5,nice app,5,2024-08-23 19:22:59
3,372a4096-ee6a-4b94-b046-cef0b646c965,"nice, ig",5,2024-08-23 19:20:50
4,b0d66a4b-9bde-4b7c-8b11-66ed6ccdd7da,"this is a great app, the bot is so accurate to...",5,2024-08-23 19:20:39


In [18]:
#let's look if dataset contain any null values
data.isnull().sum()

,0
Review Id,0
Review,6
Ratings,0
Review Date,0


In [19]:
#replace null values so it won't effect for further analysis
data['Review'] = data['Review'].astype(str).fillna('')

Now, to proceed further, we will add sentiment labels based on the review content. We will use the textblob library in Python for this task:

In [20]:
from textblob import TextBlob

# function to determine sentiment polarity
def get_sentiment(review):
    sentiment = TextBlob(review).sentiment.polarity
    if sentiment > 0:
        return 'Positive'
    elif sentiment < 0:
        return 'Negative'
    else:
        return 'Neutral'

# apply sentiment analysis
data['Sentiment'] = data['Review'].apply(get_sentiment)
sentiment_distribution = data['Sentiment'].value_counts()

Now, Look at the dissemination of positive, neutral, and negative reviews:

In [21]:
fig = go.Figure(data=[go.Bar(
    x=sentiment_distribution.index,
    y=sentiment_distribution.values,
    marker_color=['green', 'gray', 'red'],
)])

fig.update_layout(
    title='Sentiment Dissemination of ChatGPT Reviews',
    xaxis_title='Sentiment',
    yaxis_title='Number of Reviews',
    width=800,
    height=600
)

fig.show()

Most reviews are positive, with fewer neutral or negative ones. This might indicate that most users tend to hold opinions that the ChatGPT service is good though there will always be a significant number of neutral and negative reviews.


**Discussion of What Users Like about ChatGPT:**

In deducing what users like in ChatGPT, I would look at extracting common phrases and keywords from the reviews that have a positive sentiment since it might help in identifying recurring themes or features that users like.

I’ll proceed with the following steps:

1.I will filter out the reviews that have a positive sentiment.




2.Extract and comment on the most prevailing phrases and words in the positive reviews.

In [22]:
# filter reviews with positive sentiment
positive_reviews = data[data['Sentiment'] == 'Positive']['Review']

# use CountVectorizer to extract common phrases (n-grams)
vectorizer = CountVectorizer(ngram_range=(2, 3), stop_words='english', max_features=100)
X = vectorizer.fit_transform(positive_reviews)

# sum the counts of each phrase
phrase_counts = X.sum(axis=0)
phrases = vectorizer.get_feature_names_out()
phrase_freq = [(phrases[i], phrase_counts[0, i]) for i in range(len(phrases))]

# sort phrases by frequency
phrase_freq = sorted(phrase_freq, key=lambda x: x[1], reverse=True)

phrase_data = pd.DataFrame(phrase_freq, columns=['Phrase', 'Frequency'])

fig = px.bar(phrase_data,
             x='Frequency',
             y='Phrase',
             orientation='h',
             title='Top Common Phrases in Positive Reviews',
             labels={'Phrase': 'Phrase', 'Frequency': 'Frequency'},
             width=1000,
             height=600)

fig.update_layout(
    xaxis_title='Frequency',
    yaxis_title='Phrase',
    yaxis={'categoryorder':'total ascending'}
)

fig.show()

The visualization highlights that users appreciate ChatGPT for being a “great app”, with phrases like “amazing app”, “AI app”, and “excellent app” frequently mentioned in positive reviews. Users find it “useful”, “user-friendly”, and helpful for students, with many praising its AI capabilities. Additionally, the app’s ability to answer questions effectively and its free version is also valued by users.


These sentiments suggest that in terms of usability, educational value, and AI-based, the user experience that ChatGPT has is to be cherished.

**Dicussion of What Users Do Not Like About ChatGPT:**

Similarly, to analyze what users do not like about ChatGPT, I will focus on extracting common phrases and keywords from the reviews having negative sentiments. This will help in identifying which issues or features are repetitive in terms of dissatisfaction from the users' side.


I’ll proceed with the following steps:

1.Filtering the reviews with negative sentiments.






2.Extraction and analysis of the most significant phrases and keywords in these
negative reviews.

In [23]:
# filter reviews with negative sentiment
negative_reviews = data[data['Sentiment'] == 'Negative']['Review']

# use CountVectorizer to extract common phrases (n-grams) for negative reviews
X_neg = vectorizer.fit_transform(negative_reviews)

# sum the counts of each phrase in negative reviews
phrase_counts_neg = X_neg.sum(axis=0)
phrases_neg = vectorizer.get_feature_names_out()
phrase_freq_neg = [(phrases_neg[i], phrase_counts_neg[0, i]) for i in range(len(phrases_neg))]

# sort phrases by frequency
phrase_freq_neg = sorted(phrase_freq_neg, key=lambda x: x[1], reverse=True)

phrase_neg_data = pd.DataFrame(phrase_freq_neg, columns=['Phrase', 'Frequency'])

fig = px.bar(phrase_neg_data,
             x='Frequency',
             y='Phrase',
             orientation='h',
             title='Top Common Phrases in Negative Reviews',
             labels={'Phrase': 'Phrase', 'Frequency': 'Frequency'},
             width=1000,
             height=600)

fig.update_layout(
    xaxis_title='Frequency',
    yaxis_title='Phrase',
    yaxis={'categoryorder':'total ascending'}
)

fig.show()

The visualization shows a problem where the users of ChatGPT complaint about how it is a "bad app" or "useless app," said that the application contains faults or wrong information, such as "doesn't work", "error occurred", and "network error", which is contained in various negative reviews. Users also complain about technical issues, such as "error messages," and failed to use specific features, like "voice chat.".

Problems like these bring to light that users are not happy with the reliability, accuracy, and overall performance of the app.

**common problems faced by users in ChatGpt:**

I will start to analyze some of the common issues people have experienced with ChatGPT by trending the common phrases and keywords from the negative reviews. I will classify these phrases under broader problem areas, as well as then visualize which are the most common issues. Let's get started with it:

In [24]:
# grouping similar phrases into broader problem categories
problem_keywords = {
    'Incorrect Answers': ['wrong answer', 'gives wrong', 'incorrect', 'inaccurate', 'wrong'],
    'App Performance': ['slow', 'lag', 'crash', 'bug', 'freeze', 'loading', 'glitch', 'worst app', 'bad app', 'horrible', 'terrible'],
    'User Interface': ['interface', 'UI', 'difficult to use', 'confusing', 'layout'],
    'Features Missing/Not Working': ['feature missing', 'not working', 'missing', 'broken', 'not available'],
    'Quality of Responses': ['bad response', 'useless', 'poor quality', 'irrelevant', 'nonsense']
}

# initialize a dictionary to count problems
problem_counts = {key: 0 for key in problem_keywords.keys()}

# count occurrences of problem-related phrases in negative reviews
for phrase, count in phrase_freq_neg:
    for problem, keywords in problem_keywords.items():
        if any(keyword in phrase for keyword in keywords):
            problem_counts[problem] += count
            break

problem_data = pd.DataFrame(list(problem_counts.items()), columns=['Problem', 'Frequency'])

fig = px.bar(problem_data,
             x='Frequency',
             y='Problem',
             orientation='h',
             title='Common Problems Faced by Users in ChatGPT',
             labels={'Problem': 'Problem', 'Frequency': 'Frequency'},
             width=1000,
             height=600)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis_title='Frequency',
    yaxis_title='Problem',
    yaxis={'categoryorder':'total ascending'}
)

fig.show()

The visualization indicates that "Incorrect Answers" were the most common issues encountered by users when using ChatGPT, at a substantial difference from the next closest competitor. Other major complaints dealt with "App Performance," in which users suffered problems in the app running smooth and functioning properly, and "Quality of Responses."

These would point out specific areas to be improved so it may become better for the user experience through the use of ChatGPT.


**Looking into how Reviews Evolved Over Time:**


Now, I'll analyze the trend of reviews. To understand how the reviews have evolved in terms of time, I will:




1.Aggregate reviews by their sentiment (Positive, Neutral, Negative) over time.




2.Imagine the time-series patterns of the volumes for each type of sentiment.

In [25]:
# convert 'Review Date' to datetime format
data['Review Date'] = pd.to_datetime(data['Review Date'])

# aggregate sentiment counts by date
sentiment_over_time = data.groupby([data['Review Date'].dt.to_period('M'), 'Sentiment']).size().unstack(fill_value=0)

# convert the period back to datetime for plotting
sentiment_over_time.index = sentiment_over_time.index.to_timestamp()

fig = go.Figure()

fig.add_trace(go.Scatter(x=sentiment_over_time.index, y=sentiment_over_time['Positive'],
                         mode='lines', name='Positive', line=dict(color='green')))
fig.add_trace(go.Scatter(x=sentiment_over_time.index, y=sentiment_over_time['Neutral'],
                         mode='lines', name='Neutral', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=sentiment_over_time.index, y=sentiment_over_time['Negative'],
                         mode='lines', name='Negative', line=dict(color='red')))

fig.update_layout(
    title='Sentiment Trends Over Time',
    xaxis_title='Date',
    yaxis_title='Number of Reviews',
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title_text='Sentiment',
    xaxis=dict(showgrid=True, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridcolor='lightgray')
)

fig.show()

The graph illustrates the growth of positively reviewed mentions of ChatGPT, which increased over time but notably surged in March 2024 and reached the top in May 2024 and then declined a little bit in July 2024.

Even for neutral reviews (grey color line), an increase in mentions in general was observed over time, which grew and then peaked noticeably around May 2024.

Negative reviews have been stable and minimum in the term that means although more and more people share their experience over time, the people's general attitude is still positive with slowly but surely increased negative feedback.

**Analysing how often users Recommend ChatGpt:**

To conclude this task on the analysis of reviews for ChatGPT, we will analyze how often users recommend ChatGPT. Here, we shall use the concept of the Net Promoter Score. The NPS is a widely used metric to measure customer satisfaction and loyalty through the likeliness of users recommending a product or service. NPS is usually calculated using responses to the question: "How likely are you to recommend our product to a friend or colleague?"


on a scale from 0 to 10. Here's how NPS is calculated:


* Promoters: Respondents who score 9 or 10

* Passives: Respondents who score 7 or 8

* Detractors: Respondents who score between 0 and 6


**NPS = % Promoters − % Detractors**

Under the assumption that Promoters give the app 5-star ratings and Passives
rate it 4 stars, while Detractors have a rating of 3 stars or less, let's proceed to compute NPS for this assumption.

In [26]:
# define the categories based on the ratings
data['NPS Category'] = data['Ratings'].apply(lambda x: 'Promoter' if x == 5 else ('Passive' if x == 4 else 'Detractor'))

# calculate the percentage of each category
nps_counts = data['NPS Category'].value_counts(normalize=True) * 100

# calculate NPS
nps_score = nps_counts.get('Promoter', 0) - nps_counts.get('Detractor', 0)

# display the NPS Score
nps_score

64.35313912172705

The Net Promoter Score (NPS) for ChatGPT, based on the ratings provided in the dataset, is approximately 64.35. It indicates a strong likelihood that users would recommend ChatGPT to others, as a score above 50 is generally considered excellent.

#**Summary:**

So, that was just an overview of how one could analyze the reviews of a product or service. Analyzing reviews is one of the best data-driven strategies to improve your product or services. It helps understand what people think about your product, what they do not like about it, and what more they expect from it. I hope you liked this article on ChatGPT reviews analysis with Python.